In [1]:
from selenium import webdriver
import numpy as np
import os
import shutil
import time

In [2]:
url = 'http://www2.sefaz.ce.gov.br/alfrescoPublic/br.com.alfresco.FormMain/FormMain.html'

#Variavel que recebe as configuracoes do chrome
options= webdriver.ChromeOptions()

# Caminho para onde será baixado os dados
path = r'C:\Users\willi\Documents\MineracaoSEFAZ\data'

#Pasta onde será realizado o download dos arquivos
prefs = {'download.default_directory' : path,'directory_upgrade' : True}

#Cria uma configuracao do chrome com a pasta de destino especificada para donwload
options.add_experimental_option('prefs', prefs)

In [3]:
# Inicializa o browser de teste
# Coloque o caminho para onde está o seu driver do Chrome na variável abaixo
pathDriver = 'D:/ChromeDriver/chromedriver'
browser = webdriver.Chrome(pathDriver, options = options)

#Abre o site da SEFAZ 
browser.get(url)

In [4]:
def get_names(isRoot):
    names = browser.find_elements_by_xpath("//td[@align = 'left']//div[@cellclipdiv = 'true']")
    names_content = [x.get_attribute('innerHTML').strip() for i,x in enumerate (names) if i%2==0]
    names = [x for i,x in enumerate (names) if i%2==0]
    
    if isRoot == 1:
        names = names[1:]
        names_content = names_content[1:]
    
    return names,names_content
    
    
def back(sleep = 1):
    back = browser.find_element_by_xpath('//div[contains(text(), "' + 'Retorna ao nível superior' + '")]')
    try:
        
        back.click()
    except:
        
        back.location_once_scrolled_into_view
        time.sleep(sleep)
        back.click()
    

In [5]:
def downloader(isRoot = 0,path = '',file = 'null.txt'):
    names,_ = get_names(isRoot)

    num = len(names)
    sleep = 1 + num/10 #valor para a volta
    
    if (num == 0) and (isRoot == 1): #Trata exceção de página vazia
        back()
    
    for i in range (num):
        names,names_content = get_names(isRoot)
        name,name_content = names[i],names_content[i]
        
        if isRoot == 0:
            file_name = path + '/' + name_content + '.txt'
            
            if os.path.exists(file_name):
                continue
                
            file = open(file_name,'a')
        else :
            file = file 
            file.write('->' + name_content + '\n')
        
        try:
            name.click()
            
        except:
            name.location_once_scrolled_into_view
            time.sleep(2 + i/10)
            name.click()
            
        time.sleep(2 + i/10) # Tempo para entrar na página 
        
        search_downloads = search_downloads = browser.find_elements_by_name('download')
        
        if len(search_downloads) == 0:
            downloader(isRoot = 1,file = file)
        
        else :
            pdf_elems = browser.find_elements_by_xpath("//a[contains(@href,'www')]")
            pdf_links = [x.get_attribute('href').strip() for x in pdf_elems]
            _,pdf_names = get_names(1)
            
            sleep = 1 + (len(pdf_links))/10 #valor para a volta
            
            for i in  range (len(pdf_links)) :
                file.write(pdf_names[i] + '-->' + pdf_links[i] + '\n')
        
        back(sleep = sleep)
        
        if isRoot == 0:
            file.close()
        
        time.sleep(3) # Tempo para voltar para a páginas das pastas
            
        
        

In [6]:
downloader(path=path)